In [33]:
# ruff: noqa: N802, N803, N806, N815, N816
import os
import pprint

import numpy as np
from scipy import signal

# Simple utilities for displaying generated code in the notebook
from utils import cleanup, display_text

import archimedes as arc
from archimedes import struct

THEME = os.environ.get("ARCHIMEDES_THEME", "dark")
arc.theme.set_theme(THEME)

# Structured data types

So far we have been working with a function that operates with regular arrays:

In [ ]:
@arc.compile(return_names=("u_hist", "y_hist"))
def iir_filter(u, b, a, u_prev, y_prev):
    # Update input history
    u_prev[1:] = u_prev[:-1]
    u_prev[0] = u

    # Compute output using the direct II transposed structure
    y = (np.dot(b, u_prev) - np.dot(a[1:], y_prev[: len(a) - 1])) / a[0]

    # Update output history
    y_prev[1:] = y_prev[:-1]
    y_prev[0] = y

    return u_prev, y_prev

However, this approach of passing each individual input, output, or state component as an arg doesn't scale well to more complex algorithms.
Additionally, for stateful functions it requires manual state management:

```c
// Copy output arrays back to inputs
for (int j = 0; j < n; j++) {
    arg.u_prev[j] = res.u_hist[j];
    arg.y_prev[j] = res.y_hist[j];
}
arg.u_prev[n] = res.u_hist[n];
```

This single filter uses five arguments; even if we just added two extra IIR filters, we might need as many as 15 arguments to keep track of the filter histories, coefficients, etc.
This quickly becomes difficult to understand and maintain.

One solution is to use hierarchical data structures to organize the arguments and returns.
We call these "PyTrees", using terminology borrowed from [JAX](https://docs.jax.dev/en/latest/pytrees.html).
These let you organize state and parameters in dataclass-like containers which can be mapped directly to a C `struct`.
They can even be nested inside of one another to arbitrary depth.
See the overview of [PyTrees](../../pytrees.md) or the documentation for the [`@pytree_node`](#archimedes.tree.struct.pytree_node) decorator for more details.

For example, let's say we have an algorithm that is composed of three discrete transfer functions, implemented as IIR filters:

```raw
                  |--> G --> y_g
u --> F --> y_f --|
                  |--> H --> y_h
```

Instead of maintaining six arrays (one each for `u` and `y` histories, across three filters), we can organize these into a logical hierarchical structure as follows:

In [ ]:
@struct.pytree_node
class FilterState:
    u_prev: np.ndarray
    y_prev: np.ndarray


@struct.pytree_node
class CompoundState:
    x_f: FilterState
    x_g: FilterState
    x_h: FilterState


@struct.pytree_node
class CompoundOutput:
    y_f: float
    y_g: float
    y_h: float


# Rewrite the IIR filter to operate with the structured state
def iir_filter(
    x: FilterState, u: float, b: np.ndarray, a: np.ndarray
) -> tuple[FilterState, float]:
    u_prev, y_prev = x.u_prev, x.y_prev

    # Update input history
    u_prev[1:] = u_prev[:-1]
    u_prev[0] = u

    # Compute output using the direct II transposed structure
    y = (np.dot(b, u_prev) - np.dot(a[1:], y_prev[: len(a) - 1])) / a[0]

    # Update output history
    y_prev[1:] = y_prev[:-1]
    y_prev[0] = y

    return FilterState(u_prev, y_prev), y


@arc.compile(return_names=("state_new", "y"))
def compound_filter(
    state: CompoundState, u: float
) -> tuple[CompoundState, CompoundOutput]:
    # For simplicity, just re-use the coefficients from the low-pass filter
    # we've already designed
    x_f, y_f = iir_filter(state.x_f, u, b, a)
    x_g, y_g = iir_filter(state.x_g, y_f, b, a)
    x_h, y_h = iir_filter(state.x_h, y_f, b, a)

    return CompoundState(x_f, x_g, x_h), CompoundOutput(y_f, y_g, y_h)


x0 = CompoundState(
    x_f=FilterState(u_prev=np.zeros(len(b)), y_prev=np.zeros(len(a) - 1)),
    x_g=FilterState(u_prev=np.zeros(len(b)), y_prev=np.zeros(len(a) - 1)),
    x_h=FilterState(u_prev=np.zeros(len(b)), y_prev=np.zeros(len(a) - 1)),
)
u0 = 1.0

args = (x0, u0)
x, y = compound_filter(*args)

pprint.pprint(y)

In [ ]:
cleanup()  # Clean up any previous generated code
arc.codegen(compound_filter, args)

In [ ]:
with open("compound_filter.h", "r") as f:
    c_code = f.read()

display_text(c_code)

We see that the `FilterState` struct is automatically translated to `filter_state_t`, and the `CompoundState` and `CompoundOutput` structs are likewise translated to `compound_state_t` and `compound_output_t`.

These can be nested in one another just as in the Python code, so for example a `compound_state_t` is composed of three `filter_state_t`s

For now, you still need to manually copy the data from the output state back to the inputs (though [not for long](https://github.com/PineTreeLabs/archimedes/issues/77)), but now we can naturally access the hierarchical data in the C application:

```c
// Initialize as before
compound_filter_arg_t filter_arg;
compound_filter_res_t filter_res;
compound_filter_work_t filter_work;
compound_filter_init(&filter_arg, &filter_res, &filter_work);

// Set up the inputs to the function
filter_arg.u = read_sensor();

// Access nested states if necessary
check_state(&filter_arg.state.x_f);

// Evaluate the function numerically
compound_filter_step(&filter_arg, &filter_res, &filter_work);

// Do something with the outputs
handle_outputs(filter_res.y.y_g, filter_res.y.y_h);
```

When you call the `_step` function, all that happens is that the pointers to the underlying data are "marshalled" into a pointer array and passed to CasADi, amounting to minimal overhead compared to the gain in flexibility and readability for more complex functions.

With this approach you can create maintainable and scalable data types that match how you think about parameters, plant models, and control algorithms.

## Summary

In this final part of the hardware deployment tutorial, we've finally put the pieces together to  **TODO: Finish this**

This Python-to-C code generation provides a convenient workflow for writing high-level logic in Python and then rapidly deploying to a range of C environments.

### Where to Go From Here

As mentioned, this code generation and hardware deployment workflow is an early proof of concept, but it is central to the Archimedes roadmap.

**If you're using Archimedes for hardware control and have questions, comments, or requests, please don't hesitate to reach out!**